# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import numpy as np
import random

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
df_group = data.groupby(["CustomerID","ProductName"],as_index=True).agg({"Quantity":sum})
df_group.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
df_pivot = pd.pivot_table(df_group, values='Quantity', index="ProductName",columns="CustomerID", aggfunc=np.sum).fillna(0)
df_pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
distances = pdist(df_pivot.T, 'euclidean')
distances_matrix = squareform(distances)

distances = pd.DataFrame(1/(1 + distances_matrix), index=df_pivot.columns, columns=df_pivot.columns)
display(distances.head())

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
results = list(distances[33].sort_values(ascending=False).index[1:6])
results

[264, 3535, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [8]:
similar_customers = df_group.loc[results]
similar_customers.head()

Quantity
CustomerID ProductName                         
264        Apricots - Halves                  1
           Apricots Fresh                     1
           Bacardi Breezer - Tropical         1
           Bagel - Plain                      1
           Banana - Leaves                    1

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [9]:
ranked_products = similar_customers.groupby(["ProductName"],as_index=True).agg({"Quantity":sum}).sort_values("Quantity",ascending=False)
ranked_products.head()

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [10]:
top_5 = pd.merge(df_pivot, ranked_products, on="ProductName").sort_values(by="Quantity",ascending=False)
list_5 = list(top_5[top_5[33]==0][33].index)[:5]
list_5

['Butter - Unsalted',
 'Wine - Ej Gallo Sierra Valley',
 'Soup - Campbells Bean Medley',
 'Wine - Blue Nun Qualitatswein',
 'Chicken - Soup Base']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [11]:
recommendations = {"CustomerID":[],"Product_1":[],"Product_2":[],"Product_3":[],"Product_4":[],"Product_5":[]}
customers = list(distances.keys())

for customer in customers:
    recommendations["CustomerID"].append(customer)
    results = list(distances[customer].sort_values(ascending=False).index[1:6])
    similar_customers = df_group.loc[results]
    ranked_products = similar_customers.groupby(["ProductName"],as_index=True).agg({"Quantity":sum}).sort_values("Quantity",ascending=False)
    top_5 = pd.merge(df_pivot, ranked_products, on="ProductName").sort_values(by="Quantity",ascending=False)
    list_5 = list(top_5[top_5[customer]==0][customer].index)[:5]
    recommendations["Product_1"].append(list_5[0])
    recommendations["Product_2"].append(list_5[1])
    recommendations["Product_3"].append(list_5[2])
    recommendations["Product_4"].append(list_5[3])
    recommendations["Product_5"].append(list_5[4])

##  Step 9: Store the results in a Pandas data frame. The data frame should have a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [12]:
result = pd.DataFrame(recommendations)
result.head(20)

,CustomerID,Product_1,Product_2,Product_3,Product_4,Product_5
0,33,Butter - Unsalted,Wine - Ej Gallo Sierra Valley,Soup - Campbells Bean Medley,Wine - Blue Nun Qualitatswein,Chicken - Soup Base
1,200,Soup - Campbells Bean Medley,Muffin - Carrot Individual Wrap,Bay Leaf,Pork - Kidney,Wanton Wrap
2,264,Soupfoamcont12oz 112con,Wine - Two Oceans Cabernet,Bread - Italian Roll With Herbs,"Veal - Inside, Choice","Fish - Scallops, Cold Smoked"
3,356,"Veal - Inside, Choice",Lamb - Ground,Pomello,Butter - Unsalted,"Tart Shells - Sweet, 4"
4,412,Olive - Spread Tapenade,Sprouts - Baby Pea Tendrils,Wine - Blue Nun Qualitatswein,"Pepper - Black, Whole",Soup - Campbells Bean Medley
5,464,Butter - Unsalted,"Sauce - Gravy, Au Jus, Mix","Pepper - Black, Whole","Peas - Pigeon, Dry",Bar - Granola Trail Mix Fruit Nut
6,477,Olive - Spread Tapenade,Sprouts - Baby Pea Tendrils,Wine - Blue Nun Qualitatswein,"Veal - Inside, Choice","Pepper - Black, Whole"
7,639,Cod - Black Whole Fillet,Wine - Ej Gallo Sierra Valley,Yoghurt Tubes,"Pepper - Black, Whole",Soupfoamcont12oz 112con
8,649,Bar - Granola Trail Mix Fruit Nut,Wine - Magnotta - Belpaese,Appetizer - Mushroom Tart,"Peas - Pigeon, Dry","Pepper - Black, Whole"
9,669,"Veal - Inside, Choice",Sardines,Wine - Ej Gallo Sierra Valley,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [13]:
distances_2 = pdist(df_pivot.T, 'cosine')
distances_matrix = squareform(distances_2)
distances_2 = pd.DataFrame(1/(1 + distances_matrix), index=df_pivot.columns, columns=df_pivot.columns)

recommendations_2 = {"CustomerID":[],"Product_1":[],"Product_2":[],"Product_3":[],"Product_4":[],"Product_5":[]}

for customer in customers:
    recommendations_2["CustomerID"].append(customer)
    results = list(distances_2[customer].sort_values(ascending=False).index[1:6])
    similar_customers = df_group.loc[results]
    ranked_products = similar_customers.groupby(["ProductName"],as_index=True).agg({"Quantity":sum}).sort_values("Quantity",ascending=False)
    top_5 = pd.merge(df_pivot, ranked_products, on="ProductName").sort_values(by="Quantity",ascending=False)
    list_5 = list(top_5[top_5[customer]==0][customer].index)[:5]
    recommendations_2["Product_1"].append(list_5[0])
    recommendations_2["Product_2"].append(list_5[1])
    recommendations_2["Product_3"].append(list_5[2])
    recommendations_2["Product_4"].append(list_5[3])
    recommendations_2["Product_5"].append(list_5[4])   

In [14]:
result_2 = pd.DataFrame(recommendations_2)
display(result_2.head(20))

,CustomerID,Product_1,Product_2,Product_3,Product_4,Product_5
0,33,Knife Plastic - White,"Soup - Campbells, Beef Barley",Onions - Cippolini,Tea - Herbal Sweet Dreams,Muffin - Zero Transfat
1,200,Longos - Grilled Salmon With Bbq,Snapple Lemon Tea,General Purpose Trigger,"Thyme - Lemon, Fresh",Tomatoes Tear Drop
2,264,Pickerel - Fillets,"Water - Mineral, Natural",Snapple - Iced Tea Peach,Wine - Ej Gallo Sierra Valley,French Pastry - Mini Chocolate
3,356,Bread - English Muffin,Olive - Spread Tapenade,Bagel - Plain,Pork - Hock And Feet Attached,Wiberg Super Cure
4,412,"Salsify, Organic",Durian Fruit,Wine - Hardys Bankside Shiraz,Bread - Raisin Walnut Oval,Gatorade - Xfactor Berry
5,464,Wonton Wrappers,Bread - Bistro White,Sausage - Breakfast,"Tart Shells - Sweet, 4","Juice - Cranberry, 341 Ml"
6,477,"Pasta - Penne, Rigate, Dry",Olive - Spread Tapenade,Spinach - Baby,Veal - Osso Bucco,Carbonated Water - Blackcherry
7,639,Wine - Pinot Noir Latour,Crab - Imitation Flakes,Cheese - Parmesan Cubes,Bouq All Italian - Primerba,Whmis - Spray Bottle Trigger
8,649,"Fish - Scallops, Cold Smoked",Napkin White - Starched,"Mushroom - Porcini, Dry",Wine - Pinot Noir Latour,Pork - Inside
9,669,Cookies - Assorted,Wine - Blue Nun Qualitatswein,Pickerel - Fillets,Beer - Rickards Red,"Hickory Smoke, Liquid"
